In [3]:
# Import necessary libraries
import pandas as pd

# Load the original 2022 and 2024 datasets
data_2022 = pd.read_stata('S://biostats_consulting_lab//2022SurveyedPeople.dta')
data_2024 = pd.read_stata('S://biostats_consulting_lab//2024SurveyedPeople.dta')

In [4]:
# Data Cleaning
## 1.1 Standardize date formats
data_2022['sec1_q1'] = pd.to_datetime(data_2022['sec1_q1'], errors='coerce')  # Date column in 2022 data
data_2024['birthdate'] = pd.to_datetime(data_2024['birthdate'], format='%d%b%Y', errors='coerce')  # Birthdate column in 2024 data

## 1.2 Ensure categorical variables are handled correctly
categorical_vars_2022 = ['sec1_q4', 'sec1_q5', 'sec1_q6', 'sec1_q7', 'sec1_q9', 'sec11_q156', 'sec11_q158']
categorical_vars_2024 = ['gender', 'educ_level', 'employ_status', 'hh_income', 'religion', 'call_status']

# Convert columns to categorical data type
for var in categorical_vars_2022:
    data_2022[var] = pd.Categorical(data_2022[var])

for var in categorical_vars_2024:
    data_2024[var] = pd.Categorical(data_2024[var])

In [7]:
# Generate codebook
## Load the 2024 codebook and 2022 baseline metadata
codebook_2024 = pd.read_excel('S://biostats_consulting_lab//HPVax_FU_phonesurvey_Deidentified_codebook.xlsx', sheet_name='Sheet1')
baseline_metadata = pd.read_excel('S://biostats_consulting_lab//HPVax_Baseline_Main_Survey_VO.xlsx', sheet_name='survey')

# Create mappings of field names to their descriptions
label_mapping_2022 = baseline_metadata[['name', 'label']].dropna().set_index('name')['label'].to_dict()
label_mapping_2024 = codebook_2024[['Variable Name', 'Variable Label']].dropna().set_index('Variable Name')['Variable Label'].to_dict()

# Function to extract coding information for categorical variables
def extract_coding_info(df):
    coding_info = {}
    for col in df.columns:
        if pd.api.types.is_categorical_dtype(df[col]):
            coding_info[col] = {val: df[col].cat.categories[val] for val in range(len(df[col].cat.categories))}
        else:
            coding_info[col] = 'Not categorical / No coding available'
    return coding_info

# Generate codebook including field descriptions and coding
coding_info_2022 = extract_coding_info(data_2022)
coding_info_2024 = extract_coding_info(data_2024)

codebook_2022 = pd.DataFrame({
    'Variable Name': data_2022.columns,
    'Description': [label_mapping_2022.get(col, 'No description available') for col in data_2022.columns],
    'Coding': [coding_info_2022.get(col, 'Not available') for col in data_2022.columns]
})

codebook_2024 = pd.DataFrame({
    'Variable Name': data_2024.columns,
    'Description': [label_mapping_2024.get(col, 'No description available') for col in data_2024.columns],
    'Coding': [coding_info_2024.get(col, 'Not available') for col in data_2024.columns]
})

C:\Users\ghlas\AppData\Local\Temp\ipykernel_21824\1452867160.py:14: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df[col]):


In [9]:
# Validate data integrity
## Check unique values of categorical variables
unique_values_2022 = {
    'Gender': data_2022['sec1_q4'].unique(),
    'Education Level': data_2022['sec1_q5'].unique(),
    'Employment Status': data_2022['sec1_q6'].unique(),
    'Marital Status': data_2022['sec1_q7'].unique()
}

unique_values_2024 = {
    'Gender': data_2024['gender'].unique(),
    'Education Level': data_2024['educ_level'].unique(),
    'Employment Status': data_2024['employ_status'].unique(),
    'Marital Status': data_2024['marital_status'].unique()
}

# Print unique values to validate categorical variables
print("Unique values in 2022 dataset:", unique_values_2022)
print("Unique values in 2024 dataset:", unique_values_2024)

# Random sampling to check if data is correctly mapped to fields
sample_2022 = data_2022.sample(5)
sample_2024 = data_2024.sample(5)

print("Random sample from 2022 dataset:\n", sample_2022)
print("Random sample from 2024 dataset:\n", sample_2024)

Unique values in 2022 dataset: {'Gender': ['Male', 'Female']
Categories (2, object): ['Male' < 'Female'], 'Education Level': ['Secondary', 'Primary', 'Higher', 'No school/Did not complete primary']
Categories (4, object): ['Primary' < 'Secondary' < 'Higher' < 'No school/Did not complete primary'], 'Employment Status': ['Casual laborer', 'Not employed and not looking for work', 'Not employed but looking for work', 'Self-employed', 'Employed full-time', 'Employed part-time', 'Prefer not to answer [do not read aloud]']
Categories (7, object): ['Prefer not to answer [do not read aloud]' < 'Employed full-time' < 'Employed part-time' < 'Casual laborer' < 'Self-employed' < 'Not employed but looking for work' < 'Not employed and not looking for work'], 'Marital Status': ['Divorced/Separated', 'Married', 'Single', 'Widowed', 'Cohabiting/Partnered', 'Prefer not to answer [do not read aloud]']
Categories (6, object): ['Prefer not to answer [do not read aloud]' < 'Single' < 'Cohabiting/Partnered' 

In [ ]:
# Save cleaned datasets
data_2022.to_csv('cleaned_2022_survey.csv', index=False)
data_2024.to_csv('cleaned_2024_survey.csv', index=False)